In [37]:
import pandas as pd
import numpy as np
import plotly.express as px

# impute missing values with iterative imputer 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [41]:
products_data = pd.read_excel('./data/Account Products.xlsx', sheet_name='web_traffic_accounts')
products_data.Date = pd.to_datetime(products_data.Date)
products_data.set_index('Date', inplace=True)
products_data.head()

,abanca.com/es/cuentas/cuenta-online,n26.com/es-es/cuenta-sin-comisiones,bancsabadell.com/cuenta-online,ing.es/cuenta-nocuenta-ing,bbva.es/personas/productos/cuentas/cuenta-online-sin-comisiones.html,revolut.com/es-ES/a-radically-better-account/,openbank.es/cuenta-ahorro-bienvenida,myinvestor.es/cuentas-tarjetas/cuentas,n26.com/es-es/cuenta-ahorro,bankinter.com/banca/cuentas-tarjetas/cuentas/cuenta-nomina,ing.es/cuenta-nocuenta-ing.1,evobanco.com/cuenta-inteligente,bancosantander.es/particulares/cuentas-tarjetas/cuentas-corrientes/cuenta-online-sin-comisiones
Date,,,,,,,,,,,,,
2021-12-01,15202.983835,6664.368911,NaN,NaN,45449.486787,1343.418328,NaN,5319.608623,NaN,8733.560835,NaN,5055.542564,22325.885971
2022-01-01,42183.913207,5310.177360,NaN,NaN,38306.464983,1531.016801,NaN,6577.851993,NaN,11410.593443,NaN,4539.012047,28331.245044
2022-02-01,34498.153115,5547.588971,NaN,NaN,40978.325821,1304.591257,NaN,4031.716960,NaN,8891.661053,NaN,6049.457905,23798.819170
2022-03-01,34546.319021,4900.271300,NaN,NaN,43970.545627,1105.180575,NaN,4941.009311,NaN,7446.925144,NaN,9826.193608,24808.561729
2022-04-01,40508.288420,4909.581023,NaN,NaN,40782.451956,1020.968713,NaN,5774.734455,NaN,10285.861726,NaN,5028.236254,24739.758729


In [42]:
imputer = IterativeImputer(max_iter=10, random_state=0)
imputed_data = imputer.fit_transform(products_data)
imputed_data = pd.DataFrame(imputed_data, columns=products_data.columns, index=products_data.index)
products_data = imputed_data.copy()
products_data.head()

,abanca.com/es/cuentas/cuenta-online,n26.com/es-es/cuenta-sin-comisiones,bancsabadell.com/cuenta-online,ing.es/cuenta-nocuenta-ing,bbva.es/personas/productos/cuentas/cuenta-online-sin-comisiones.html,revolut.com/es-ES/a-radically-better-account/,openbank.es/cuenta-ahorro-bienvenida,myinvestor.es/cuentas-tarjetas/cuentas,n26.com/es-es/cuenta-ahorro,bankinter.com/banca/cuentas-tarjetas/cuentas/cuenta-nomina,ing.es/cuenta-nocuenta-ing.1,evobanco.com/cuenta-inteligente,bancosantander.es/particulares/cuentas-tarjetas/cuentas-corrientes/cuenta-online-sin-comisiones
Date,,,,,,,,,,,,,
2021-12-01,15202.983835,6664.368911,20611.877467,24449.393654,45449.486787,1343.418328,15254.582228,5319.608623,15239.190071,8733.560835,24448.706786,5055.542564,22325.885971
2022-01-01,42183.913207,5310.177360,28890.415605,24463.625737,38306.464983,1531.016801,17359.852958,6577.851993,11468.598706,11410.593443,24463.492621,4539.012047,28331.245044
2022-02-01,34498.153115,5547.588971,24924.681184,24456.839455,40978.325821,1304.591257,16944.054696,4031.716960,12928.637519,8891.661053,24456.441669,6049.457905,23798.819170
2022-03-01,34546.319021,4900.271300,23968.596015,24455.725835,43970.545627,1105.180575,16387.002853,4941.009311,12554.646465,7446.925144,24455.281024,9826.193608,24808.561729
2022-04-01,40508.288420,4909.581023,27150.311205,24461.184509,40782.451956,1020.968713,17414.866042,5774.734455,11887.397875,10285.861726,24460.962770,5028.236254,24739.758729


In [45]:
# get bank name from columns 
products_data.columns = products_data.columns.str.split('.').str[0]

In [2]:
data = pd.read_csv('data/site_changes.csv')
data.head()

,month,year,incentive,apr,notes,bank,notes.1,tag,site changes
0,1,2021,0€,0%,NaN,abanca,NaN,NaN,NaN
1,6,2021,150€,0%,800-1200 deposit,abanca,NaN,incentive increase,NaN
2,8,2021,150€,0%,800-1200 deposit,abanca,NaN,incentive decrease,NaN
3,4,2022,300€,0%,if over 1200 deposit,abanca,NaN,apr increase,NaN
4,12,2022,300€,0%,mejor cambia de el sitio,abanca,NaN,"incentive decrease, apr decrease",major site change


In [3]:
data.tag.value_counts()

tag
increase incentive                  12
increase apr                        12
incentive increase                   4
apr increase                         3
decrease apr                         3
incentive decrease, apr decrease     2
incentive decrease, apr increase     2
incentive decrease                   1
incentive increase, apr increase     1
decrease incentive                   1
decrease incentive                   1
temporary incentive                  1
Name: count, dtype: int64

In [10]:
# rename tags

data.tag = data.tag.replace({'apr increase':'increase apr', 'incentive increase':'increase incentive', 'decrease incentive':'incentive decrease',  'decrease incentive ':'incentive decrease'})

       
data.tag.value_counts()

tag
increase incentive                  16
increase apr                        15
incentive decrease                   3
decrease apr                         3
incentive decrease, apr decrease     2
incentive decrease, apr increase     2
incentive increase, apr increase     1
temporary incentive                  1
Name: count, dtype: int64

create two lists one for APR and Incentive increases. 

In [29]:
apr_increase_dates = data[data.tag == 'increase apr']
incentive_increase_dates = data[data.tag == 'increase incentive']

# create a date column from month and year and add day as 1
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))

all_dates = data.date.unique()



We now have to find control data for each of the banks. This would be long periods of time for each bank where there is no incentive changes. 

In [1]:
data.head(20)

NameError: name 'data' is not defined

In [34]:
banks = data.bank.unique()
banks

array(['abanca', 'banc sabadell', 'bankinter', 'bbva', 'evobanco',
       'myinvestor', 'n26', 'openbank', 'revolut', 'santander'],
      dtype=object)

In [60]:
banks[-1] = 'bancosantander'

In [61]:
# for each bank we take a subsection of the data that has no incentives 

control_data = {}

for bank in banks:
    print(bank)
    bank = bank.replace(' ', '')
    incentive_dates = data.loc[(data.bank == bank), 'date']
    control_data[bank] = products_data.loc[(~products_data.index.isin(incentive_increase_dates)), bank]

control_data
    

    

abanca
banc sabadell
bankinter
bbva
evobanco
myinvestor
n26
openbank
revolut
bancosantander


{'abanca': Date
 2021-12-01    15202.983835
 2022-01-01    42183.913207
 2022-02-01    34498.153115
 2022-03-01    34546.319021
 2022-04-01    40508.288420
 2022-05-01    29791.338180
 2022-06-01    35087.423687
 2022-07-01    61969.499304
 2022-08-01    53939.003961
 2022-09-01    50991.056414
 2022-10-01    50567.127100
 2022-11-01    54180.810911
 2022-12-01    68183.612577
 2023-01-01    72370.734829
 2023-02-01    49474.725810
 2023-03-01    57832.812519
 2023-04-01    52944.026350
 2023-05-01    57922.472646
 2023-06-01    64628.532316
 2023-07-01    77975.997321
 2023-08-01    90688.236048
 2023-09-01    69082.388420
 2023-10-01    75669.229316
 2023-11-01    60025.431952
 2023-12-01    57677.133034
 Name: abanca, dtype: float64,
 'bancsabadell': Date
 2021-12-01    20611.877467
 2022-01-01    28890.415605
 2022-02-01    24924.681184
 2022-03-01    23968.596015
 2022-04-01    27150.311205
 2022-05-01    12267.129453
 2022-06-01    27765.978171
 2022-07-01    33651.426645
 2022-0

In [ ]:
# given that there are 3 years of data, we will look for sequences that are at least 3 months long with one month on either side of the sequence with no events or incentive increases 



In [43]:
# compare the list of all_dates with an array of dates between 2021-01-01 and 2024-01-01

all_dates_comp = pd.date_range(start='2021-01-01', end='2024-01-01', freq='MS')

# find the missing dates

missing_dates = all_dates[~all_dates.isin(all_dates_comp), '']

missing_dates

/var/folders/d4/rcb0hfcs0bzgr7z0mw72lc_w0000gn/T/ipykernel_12030/2978904238.py:7: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  missing_dates = all_dates[~all_dates.isin(all_dates_comp)]


<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]